In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GCP上的端到端ML：MLOps阶段6：使用Vertex AI Prediction开始使用TensorFlow Serving

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_tf_serving.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中执行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_tf_serving.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage6/get_started_with_tf_serving.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>        
</table>
<br/><br/><br/>

概览

本教程演示了如何使用TensorFlow Serving服务二进制文件从Vertex AI端点提供预测。

### 目标

在本教程中，您将学习如何在 `Vertex AI Endpoint` 资源上使用 `Vertex AI Prediction` 和 `TensorFlow Serving` 服务二进制文件。

此教程使用以下谷歌云机器学习服务和资源：

- `Vertex AI Prediction`
- `Vertex AI Batch Prediction`
- `Vertex AI Models`
- `Vertex AI Endpoints`

执行的步骤包括：

- 从 TensorFlow Hub 下载一个预训练的图像分类模型。
- 创建一个用于接收压缩图像数据、并输出模型输入的解压缩预处理数据的服务函数。
- 将 TensorFlow Hub 模型和服务函数上传为 `Vertex AI Model` 资源。
- 创建一个 `Endpoint` 资源。
- 将 `Model` 资源部署到带有 `TensorFlow Serving` 服务二进制文件的 `Endpoint` 资源。
- 对部署到 `Endpoint` 资源的 `Model` 资源实例进行在线预测。
- 对 `Model` 资源实例进行批量预测。

数据集

本教程使用了来自 TensorFlow Hub 的预训练图像分类模型，该模型是在 ImageNet 数据集上训练的。

了解更多关于 [ResNet V2 预训练模型](https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5)。

费用

本教程使用Google Cloud的可计费组件：

* Vertex AI
* 云存储

请了解[Vertex AI价格](https://cloud.google.com/vertex-ai/pricing)和[云存储价格](https://cloud.google.com/storage/pricing)，并使用[Pricing 计算器](https://cloud.google.com/products/calculator/)根据您预计的使用情况生成成本估算。

## 安装

安装以下包以执行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q
! pip3 install --upgrade google-cloud-pipeline-components $USER_FLAG -q
! pip3 install tensorflow-hub $USER_FLAG -q

重新启动内核

在安装额外的软件包之后，您需要重新启动笔记本内核，以便它可以找到这些软件包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*如果有可能，请确保在 GPU 运行时运行此笔记本。在 Colab 中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的谷歌云项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您会获得 300 美元的免费信用以支付计算/存储成本。

2. [确保为您的项目启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下 API: Vertex AI API、Compute Engine API 和 Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您是在本地运行此笔记本，您需要安装 [Cloud SDK](https://cloud.google.com/sdk)。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保 Cloud SDK 在此笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter 在行前加有 `!` 的行作为 shell 命令运行，并且插入以 `$` 为前缀的 Python 变量。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用 `gcloud` 获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改“REGION”变量，该变量用于笔记本的其余部分操作。以下是Vertex AI支持的区域。我们建议您选择最接近您的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能无法使用多区域存储桶进行Vertex AI的训练。并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您正在进行实时教程会话，您可能正在使用共享的测试帐户或项目。为了避免在创建的资源上发生名称冲突，您为每个实例会话创建一个时间戳，并将时间戳附加到您在此教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的谷歌云账号

**如果您正在使用Vertex AI Workbench笔记本**，您的环境已经通过身份验证。请跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并按提示通过oAuth验证您的账户。

**否则**，请按照以下步骤操作：

在Cloud Console中，转到[创建服务账户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务账户**。

在**服务帐户名称**字段中输入一个名称，然后点击**创建**。

在**授予此服务帐户对项目的访问权限**部分，点击角色下拉列表。在筛选框中输入"Vertex"，并选择**Vertex管理员**。在筛选框中输入"Storage Object Admin"，并选择**Storage Object Admin**。

点击创建。包含您的密钥的JSON文件将下载到您的本地环境。

在下面的单元格中将您的服务账户密钥路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = False
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        IS_COLAB = True
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论你使用哪种笔记本环境，都需要执行以下步骤。**

当您为Python初始化Vertex AI SDK时，需要指定一个云存储暂存桶。暂存桶是您的数据集和模型资源相关数据在会话间保留的地方。

请在下面输入您的云存储桶名称。存储桶名称必须在所有Google Cloud项目中是全局唯一的，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = "gs://" + BUCKET_NAME

只有当您的存储桶不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查云存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

配置变量

接下来，设置一些在教程中使用的变量。
导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip
import tensorflow as tf
import tensorflow_hub as hub

### 初始化 Vertex AI SDK for Python

为您的项目和相应的存储桶初始化 Vertex AI SDK for Python。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### 设置硬件加速器

您可以为训练和预测设置硬件加速器。

将变量 `DEPLOY_GPU/DEPLOY_NGPU` 设置为使用支持 GPU 的容器镜像，并指定分配给虚拟机实例的 GPU 数量。例如，要使用具有 4 个 Nvidia Telsa K80 GPU 的 GPU 容器镜像分配到每个 VM，您可以指定：

(aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

否则，指定 `(None, None)` 以在 CPU 上运行容器镜像。

了解有关您所在地区的[硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

*注*: 在 2.3 之前的 TF 版本中，对 GPU 的支持会导致此教程中的自定义模型加载失败。这是一个已知问题，在 TF 2.3 中已修复。这是由于在服务函数中生成的静态图操作导致的。如果您在自己的自定义模型中遇到此问题，请使用支持 GPU 的 TF 2.3 容器镜像。

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### 设置机器类型

接下来，设置用于预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置用于预测的虚拟机的计算资源。
- `机器类型`
     - `n1-standard`：每个vCPU的内存为3.75GB。
     - `n1-highmem`：每个vCPU的内存为6.5GB。
     - `n1-highcpu`：每个vCPU的内存为0.9GB。
 - `vCPUs`：\[2, 4, 8, 16, 32, 64, 96\]个CPU核心数

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", DEPLOY_COMPUTE)

### 启用Artifact Registry API

您必须为您的项目启用Artifact Registry API服务。

了解更多关于[启用服务](https://cloud.google.com/artifact-registry/docs/enable-service)。

In [ ]:
! gcloud services enable artifactregistry.googleapis.com

创建私有的 Docker 仓库

第一步是在 Google Artifact 存储库中创建自己的 Docker 仓库。

1. 运行 `gcloud artifacts repositories create` 命令，使用您的区域创建一个新的 Docker 仓库，描述为"docker 仓库"。

2. 运行 `gcloud artifacts repositories list` 命令，验证您的仓库是否已创建。

In [ ]:
PRIVATE_REPO = "my-docker-repo"

! gcloud artifacts repositories create {PRIVATE_REPO} --repository-format=docker --location={REGION} --description="Docker repository"

! gcloud artifacts repositories list

### 配置私有存储库的身份验证

在推送或拉取容器镜像之前，请配置Docker使用`gcloud`命令行工具对您区域的`Artifact Registry`进行身份验证请求。

In [ ]:
! gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

用于服务的容器（Docker）映像

设置用于服务预测的TensorFlow Serving Docker容器映像。

1. 从Docker Hub下载相应的CPU或GPU用于TF Serving的Docker映像。
2. 为映像创建一个标记，以便在Artifact Registry中注册。
3. 使用Artifact Registry注册映像。

了解更多关于[TensorFlow Serving](https://www.tensorflow.org/tfx/serving/docker)。

In [ ]:
# Executes in Vertex AI Workbench
if DEPLOY_GPU:
    DEPLOY_IMAGE = (
        f"{REGION}-docker.pkg.dev/"
        + PROJECT_ID
        + f"/{PRIVATE_REPO}"
        + "/tf_serving:gpu"
    )
    TF_IMAGE = "tensorflow/serving:2.5.4-gpu"
else:
    DEPLOY_IMAGE = (
        f"{REGION}-docker.pkg.dev/"
        + PROJECT_ID
        + f"/{PRIVATE_REPO}"
        + "/tf_serving:cpu"
    )
    TF_IMAGE = "tensorflow/serving:2.5.4"

if not IS_COLAB:
    if DEPLOY_GPU:
        ! sudo docker pull tensorflow/serving:2.5.4-gpu
    else:
        ! sudo docker pull tensorflow/serving:2.5.4

    ! docker tag $TF_IMAGE $DEPLOY_IMAGE
    ! docker push $DEPLOY_IMAGE
else:
    # install docker daemon
    ! apt-get -qq install docker.io

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

在Colab中执行

In [ ]:
%%bash -s $IS_COLAB $DEPLOY_IMAGE $TF_IMAGE
if [ $1 == "False" ]; then
  exit 0
fi
set -x
dockerd -b none --iptables=0 -l warn &
for i in $(seq 5); do [ ! -S "/var/run/docker.sock" ] && sleep 2 || break; done
docker pull $3
docker tag tensorflow/serving $2
docker push $2
kill $(jobs -p)

从TensorFlow Hub获取预训练模型

为了演示目的，本教程使用了来自TensorFlow Hub（TFHub）的预训练模型，然后将其上传到`Vertex AI Model`资源。一旦您拥有了`Vertex AI Model`资源，该模型就可以部署到`Vertex AI Endpoint`资源。

下载预训练模型

首先，您需要从TensorFlow Hub下载预训练模型。该模型以TF.Keras层的形式下载下来。为了完成模型，在本示例中，您将使用下载下来的TFHub模型作为一层创建一个`Sequential()`模型，并指定模型的输入形状。

In [ ]:
tfhub_model = tf.keras.Sequential(
    [hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5")]
)

tfhub_model.build([None, 224, 224, 3])

tfhub_model.summary()

### 保存模型工件

此时，模型已经存储在内存中。接下来，您需要将模型工件保存到云存储位置。

*注：* 对于 TF Serving，MODEL_DIR 必须以数字为结尾的子文件夹，例如，1。

In [ ]:
MODEL_DIR = BUCKET_URI + "/model/1"
tfhub_model.save(MODEL_DIR)

上传模型以进行服务

接下来，您将从自定义作业中上传您的TF.Keras模型到Vertex `Model`服务，这将为您的自定义模型创建一个Vertex `Model`资源。在上传过程中，您需要定义一个服务函数，将数据转换为模型所期望的格式。如果您将编码数据发送给Vertex AI，则您的服务函数会确保在将数据作为输入传递给您的模型之前在模型服务器上对其进行解码。

### 服务函数如何工作

当您向在线预测服务器发送请求时，请求会被一个HTTP服务器接收。HTTP服务器会从HTTP请求内容中提取预测请求。提取的预测请求将被转发到服务函数。对于Google预构建的预测容器，请求内容将作为`tf.string`传递给服务函数。

服务函数包括两个部分：

- `预处理函数`：
  - 将输入(`tf.string`)转换为底层模型(动态图)所期望的输入形状和数据类型。
  - 执行与训练底层模型期间相同的数据预处理 -- 例如，归一化，缩放等。
- `后处理函数`：
  - 将模型输出转换为接收应用程序所期望的格式 -- 例如，压缩输出。
  - 对接受应用程序的输出进行打包 -- 例如，添加标题，构建JSON对象等。

预处理和后处理函数都会被转换为与模型融合的静态图。底层模型的输出被传递到后处理函数。后处理函数将转换/打包后的输出传递回HTTP服务器。HTTP服务器将输出作为HTTP响应内容返回。

在为TF.Keras模型构建服务函数时，您需要考虑一个问题，即它们作为静态图运行。这意味着您不能使用需要动态图的TF图操作。如果这样做，您将在服务函数的编译期间收到一个错误，指示您正在使用不受支持的EagerTensor。

### 图像数据的serving功能

#### 预处理

为了将图像传递给预测服务，您需要将压缩的（例如，JPEG）图像字节编码为 base 64 格式，这样可以使内容在通过网络传输二进制数据时不会被修改。由于部署的模型预期输入数据为原始（未压缩）字节，您需要确保 base 64 编码的数据被转换回原始字节，然后被预处理以匹配模型的输入要求，然后作为输入传递给部署的模型。

为了解决这个问题，您需要定义一个 serving 功能（`serving_fn`），并将它附加到模型作为预处理步骤。添加一个 `@tf.function` 装饰器，以便 serving 功能与底层模型融合在一起（而不是在 CPU 上游运行）。

当您发送预测或解释请求时，请求的内容将被 base 64 解码成一个Tensorflow字符串（`tf.string`），然后传递给 serving 函数（`serving_fn`）。serving 函数对 `tf.string` 进行预处理为原始（未压缩）的 numpy 字节 (`preprocess_fn`) 以匹配模型的输入要求：

- `io.decode_jpeg` - 解压缩 JPG 图像，返回一个具有三个通道（RGB）的Tensorflow张量。
- `image.convert_image_dtype` - 将整数像素值更改为浮点32位，并重新调整像素数据在0和1之间。
- `image.resize` - 调整图像的大小以匹配模型的输入形状。

此时，数据可以通过一个具体函数传递给模型（`m_call`）。serving 函数是一个静态图，而模型是一个动态图。具体函数执行了将输入数据从 serving 函数传递给模型的任务，并将从模型返回至 serving 函数的预测结果。

In [ ]:
CONCRETE_INPUT = "numpy_inputs"


def _preprocess(bytes_input):
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(224, 224))
    return resized


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    decoded_images = tf.map_fn(
        _preprocess, bytes_inputs, dtype=tf.float32, back_prop=False
    )
    return {
        CONCRETE_INPUT: decoded_images
    }  # User needs to make sure the key matches model's input


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)
    return prob


m_call = tf.function(tfhub_model.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, 224, 224, 3], dtype=tf.float32, name=CONCRETE_INPUT)]
)

tf.saved_model.save(tfhub_model, MODEL_DIR, signatures={"serving_default": serving_fn})

## 获取serving函数的签名

您可以通过重新加载模型至内存，并查询每个层的签名来获取模型的输入和输出层的签名。

对于您的目的，您需要serving函数的签名。为什么呢？当我们将数据作为HTTP请求数据包发送进行预测时，图像数据是base64编码的，而我们的TF.Keras模型需要numpy输入。您的serving函数将从base64转换为numpy数组。

在进行预测请求时，您需要将请求路由到serving函数而不是模型，因此您需要了解serving函数的输入层名称——这在您后续进行预测请求时会用到。

In [ ]:
loaded = tf.saved_model.load(MODEL_DIR)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)

### 将 TensorFlow Hub 模型上传到 `Vertex AI Model` 资源

最后，您将来自 TFHub 模型和服务功能的模型构件上传到 `Vertex AI Model` 资源中。由于您正在使用非谷歌预构建的服务二进制文件 -- 即 TensorFlow Serving，您需要指定以下额外的服务配置设置：

- `serving_container_command`：要启动的服务二进制文件 (HTTP 服务器)。
- `serving_container_args`：传递给服务二进制文件的参数。对于 TensorFlow Serving，所需的参数包括:
     - `--model_name`：要分配给模型的可读名称。
     - `--model_base_name`：在容器中存储模型构件的位置。Vertex 服务会将变量 $(AIP_STORAGE_URI) 设置为服务在容器中安装模型构件的位置。
     - `--rest_api_port`：用于发送基于 REST 的预测请求的端口。可以是 8080 或 8501（TensorFlow Serving 的默认端口）。
     - `--port`：用于发送基于 gRPC 的预测请求的端口。对于 TensorFlow Serving，应为 8500。
- `serving_container_health_route`：服务定期 ping 以验证服务二进制文件正在运行的 URL。对于 TensorFlow Serving，这将是 /v1/models/<model_name>。
- `serving_container_predict_route`：服务路由基于 REST 的预测请求的 URL。对于 TF Serving，这将是 /v1/models/[model_name]:predict。
- `serving_container_ports`: 用于 HTTP 服务器侦听请求的端口列表。

将模型上传到 Vertex Model 资源会返回一个长时间运行的操作，因为可能需要一些时间。

*注意:* 您需要从模型路径中删除末尾的数字子文件夹（例如 /1）才能上传。Vertex 服务将上传包含模型构件的子文件夹上方的父文件夹 -- 这是 TensorFlow Serving 二进制文件所期望的。

*注意:* 当您将模型构件上传到 `Vertex AI Model` 资源时，您需要指定相应的部署容器镜像。

In [ ]:
MODEL_NAME = "example_" + TIMESTAMP

model = aip.Model.upload(
    display_name="example_" + TIMESTAMP,
    artifact_uri=MODEL_DIR[:-2],
    serving_container_image_uri=DEPLOY_IMAGE,
    serving_container_health_route="/v1/models/" + MODEL_NAME,
    serving_container_predict_route="/v1/models/" + MODEL_NAME + ":predict",
    serving_container_command=["/usr/bin/tensorflow_model_server"],
    serving_container_args=[
        "--model_name=" + MODEL_NAME,
        "--model_base_path=" + "$(AIP_STORAGE_URI)",
        "--rest_api_port=8080",
        "--port=8500",
        "--file_system_poll_wait_seconds=31540000",
    ],
    serving_container_ports=[8080],
)

print(model)

创建`Endpoint`资源

您可以使用`Endpoint.create()`方法创建`Endpoint`资源。至少，您需要指定端点的显示名称。可选地，您可以指定项目和位置（区域）；否则，设置将继承您在使用`init()`方法初始化Vertex AI SDK时设置的值。

在此示例中，指定了以下参数：

- `display_name`：`Endpoint`资源的可读名称。
- `project`：您的项目ID。
- `location`：您的区域。
- `labels`：（可选）以键/值对形式为`Endpoint`定义的用户定义的元数据。

此方法返回一个`Endpoint`对象。

了解更多关于[Vertex AI Endpoints](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)。

In [ ]:
endpoint = aip.Endpoint.create(
    display_name="example_" + TIMESTAMP,
    project=PROJECT_ID,
    location=REGION,
    labels={"your_key": "your_value"},
)

print(endpoint)

将`Model`资源部署到`Endpoint`资源。

您可以将一个或多个`Vertex AI Model`资源实例部署到同一个终端节点上。每个部署的`Vertex AI Model`资源将拥有自己的用于服务二进制文件的部署容器。

*注意:* 对于这个示例，在上一步将模型工件上传到`Vertex AI Model`资源时，您指定了TFHub模型的部署容器。

在下一个示例中，您将`Vertex AI Model`资源部署到`Vertex AI Endpoint`资源。`Vertex AI Model`资源已经为其定义了部署容器镜像。要进行部署，您需要指定以下附加配置设置：

- 机器类型。
- （如果有的话）GPU 类型和数量。
- VM 实例的静态、手动或自动扩展。

在此示例中，您将使用最少指定参数来部署模型，如下所示：

- `model`: `Model`资源。
- `deployed_model_displayed_name`: 部署模型实例的人类可读名称。
- `machine_type`: 每个VM实例的机器类型。

由于需要为资源提供资源，因此这可能需要一些时间。

In [ ]:
response = endpoint.deploy(
    model=model,
    deployed_model_display_name="example_" + TIMESTAMP,
    machine_type=DEPLOY_COMPUTE,
)

print(endpoint)

### 为预测准备测试数据

接下来，您将把一个压缩的JPEG图像加载到内存中，然后对其进行base64编码。为了演示目的，您将使用Flowers数据集中的一幅图像。

In [ ]:
! gsutil cp gs://cloud-ml-data/img/flower_photos/daisy/100080576_f52e8ee070_n.jpg test.jpg

In [ ]:
import base64

with open("test.jpg", "rb") as f:
    data = f.read()
b64str = base64.b64encode(data).decode("utf-8")

### 进行预测

现在您的`Model`资源已部署到一个`Endpoint`资源，您可以通过向Endpoint资源发送预测请求来进行在线预测。

#### 请求

在这个例子中，由于您的测试项目位于Cloud Storage存储桶中，您可以使用`tf.io.gfile.Gfile()`来打开并读取图像的内容。为了将测试数据传递给预测服务，您需要将字节编码为base64格式，这样在通过网络传输二进制数据时可以确保内容不会被修改。

每个示例的格式为：

    { serving_input: { 'b64': base64编码的字节 } }

由于`predict()`方法可以接受多个项目（实例），请将您的单个测试项目作为一个包含一个测试项目的列表发送。

#### 响应

从`predict()`调用的响应是一个Python字典，包含以下条目：

- `ids`：每个预测请求的内部分配的唯一标识符。
- `predictions`：每个类别标签的预测置信度，介于0到1之间。
- `deployed_model_id`：进行预测的已部署`Model`资源的Vertex AI标识符。

In [ ]:
# The format of each instance should conform to the deployed model's prediction input schema.
instances = [{serving_input: {"b64": b64str}}]

prediction = endpoint.predict(instances=instances)

print(prediction)

## 批处理预测简介

批处理预测提供了对大量预测请求进行脱机批处理处理的能力。资源仅在批处理过程中预留，然后在批处理请求完成后取消预留。结果存储在Cloud Storage中，与在线预测不同，在在线预测中结果作为HTTP响应数据包返回。

批处理作业的输入格式取决于您的模型服务器支持的格式。首先，您的模型服务器中的Web服务器必须支持JSONL格式，该Web服务器将其转换为模型输入接口或服务功能接口直接支持的格式。对于批处理预测，此JSONL格式被称为“轴心”格式。

### 批处理预测作业的输入格式

批处理服务器接受以下输入格式：

- JSONL
- CSV
- TFRecords
- 文件列表

### 轴心格式

批处理服务器将输入格式转换为“轴心”（JSONL）格式，如下所示：

**JSONL**

每个输入行（请求）应包含一个且仅有一个有效的JSON值。

    {"values": [1, 2, 3, 4], "key": 1}
    {"values": [5, 6, 7, 8], "key": 2}

批处理服务器以同样的格式生成“轴心”数据。然后生成的“轴心”数据被包装成一个有效负载请求：

    {"instances": [
      {"values": [1, 2, 3, 4], "key": 1},
      {"values": [5, 6, 7, 8], "key": 2}
    ]}

**CSV**

第一行中的csv标题将被始终忽略。要求字符串字段明确用双引号括起来，否则将丢弃该行并将解析错误消息输出到错误文件。未加引号的值始终被传输为浮点值。

    col1,col2,col3
    1,3,"cat1"
    2,4,"cat2"

批处理服务器将每个输入行（请求）转换为JSON数组。

    {"instances": [
     [1.0,3.0,"cat1"],
     [2.0,4.0,"cat2"]
    ]}
    
**BigQuery**

每行转换为JSON数组。例如：

    [1.0,3.0,"cat1"]
    [2.0,4.0,"cat2"]
    
批处理服务器以同样的格式生成“轴心”数据。然后生成的“轴心”数据被包装成一个有效负载请求：

    {"instances": [
     [1.0,3.0,"cat1"],
     [2.0,4.0,"cat2"]
    ]}

**TFRecords**

TFRecord文件中的实例由apache_beam.io.tfrecordio模块以二进制形式读取。然后将二进制对象序列化为ASCII字符串。预测服务器负责知道解码器以恢复实例。

    {"instances": [
     {"b64","b64EncodedASCIIString"},
     {"b64","b64EncodedASCIIString"}
    ]}

**文件列表**

文件列表格式包含文件列表。在“文件列表”文件中的每一行指定一个文件路径，指定为Cloud Storage位置。

    gs://my-bucket/file1.txt
    gs://my-bucket/file2.txt

批处理服务器将文件作为二进制文件读取。然后将二进制对象序列化为ASCII字符串。

    {"instances": [
     {"b64","b64EncodedASCIIString"},
     {"b64","b64EncodedASCIIString"}
    ]}

### 创建批量输入文件

接下来，创建一个批量输入文件，将其存储在您本地的 Cloud Storage 存储桶中。对于自定义模型，您需要以 JSONL 格式格式化批量输入文件。JSONL 文件中的每个 JSON 对象条目都需要与您为在线预测请求指定的格式相同。

换句话说，在线预测和批量预测都使用相同的预测请求格式。不同之处在于，在在线预测中，您会将请求作为内存中的字典对象传递，使用 SDK 方法 `predict()`。而对于批量预测，则需要将每个预测请求（字典条目）写成一个 JSON 对象，每行一个。

字典包含以下键值对：

- `input_name`：底层模型的输入层的名称。
- `'b64'`：指示内容为 base64 编码的键。
- `content`：作为 base64 编码字符串的压缩 JPG 图像字节。

预测请求中的每个实例都是以下形式的字典条目：

                        {serving_input: {'b64': content}}

为了将图像数据传递给预测服务，您需要将字节编码为 base64 -- 这样可以确保在通过网络传输二进制数据时，内容不会被修改。

In [ ]:
# For demonstration purposes, you write the same image (instance[0]) request twice to the JSONL file.
# You will receive back two predictions, one for each instance.

import json

with open("test.jsonl", "w") as f:
    json.dump(instances[0], f)
    f.write("\n")
    json.dump(instances[0], f)

! gsutil cp test.jsonl {BUCKET_URI}/test.jsonl

### 创建批量预测请求

现在您的模型资源已经训练好了，您可以通过调用batch_predict()方法来进行批量预测，使用以下参数：

- `job_display_name`：批量预测作业的可读名称。
- `gcs_source`：一个或多个批量请求输入文件的列表。
- `gcs_destination_prefix`：用于存储批量预测结果的 Cloud Storage 位置。
- `instances_format`：输入实例的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `predictions_format`：输出预测的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `machine_type`：用于训练的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作器复制品的加速器数量。
- `sync`：如果设置为True，调用将在等待异步批处理作业完成时阻塞。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

batch_predict_job = model.batch_predict(
    job_display_name="example_" + TIMESTAMP,
    instances_format="jsonl",
    predictions_format="jsonl",
    model_parameters=None,
    gcs_source=f"{BUCKET_URI}/test.jsonl",
    gcs_destination_prefix=f"{BUCKET_URI}/results",
    machine_type=DEPLOY_COMPUTE,
    accelerator_type=DEPLOY_GPU,
    accelerator_count=DEPLOY_NGPU,
    starting_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    sync=False,
)

### 等待批量预测作业完成

接下来，等待批处理作业完成。或者，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻塞直到批量预测作业完成。

In [ ]:
batch_predict_job.wait()

获取预测结果

接下来，从已完成的批量预测作业中获取结果。

结果将被写入您在批量预测请求中指定的 Cloud Storage 输出桶。您可以调用方法iter_outputs()获取生成的包含结果的每个 Cloud Storage 文件的列表。每个文件以 JSON 格式包含一个或多个预测请求：

- `instance`：预测请求。
- `prediction`：预测响应。

In [ ]:
import json

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

## 清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
delete_bucket = True
delete_model = True
delete_endpoint = True
delete_batch_job = True

if delete_endpoint:
    try:
        endpoint.undeploy_all()
        endpoint.delete()
    except Exception as e:
        print(e)

if delete_model:
    try:
        model.delete()
    except Exception as e:
        print(e)

if delete_bucket:
    try:
        batch_predict_job.delete()
    except Exception as e:
        print(e)

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}